In [ ]:
from pathlib import Path
import scipy.io as sio
import pandas as pd
import numpy as np
import matplotlib as plt
import pursuit_functions as pursuit
import pyarrow as pa
import re

pd.set_option('display.max_columns', 100)  # Show more columns (default is 20)

%load_ext autoreload
%autoreload 2

In [ ]:
#load region files
#data_dir = Path("/Users/may/Projects/nitzPurusitData")
data_dir = Path(r"Z:\Data\Andy\nitzPurusitData")
#data_dir = Path("/Volumes/ASA_Lab/Data/Xiaoxiao/ppcRscEVCPoster/pursuitSessionFiles")


region_directories = pursuit.file_reader.load_region_files(data_dir, prefix='sl')

In [ ]:
#convert loaded region directory files into dataframes
rsc_directory = pd.DataFrame(region_directories['slRsc'])
ppc_directory = pd.DataFrame(region_directories['slPar']) 

In [ ]:
#unpack block indices
pursuit.df_utils.unpack_block_indices(rsc_directory)
pursuit.df_utils.unpack_block_indices(ppc_directory)

In [ ]:
#load pursuit session files
#data_dir2 = Path("/Users/may/Projects/nitzPurusitData/Sessions")
data_dir2 = Path(r"Z:\Data\Andy\nitzPurusitData\Sessions")

rsc_files = rsc_directory["sessFile"].unique().tolist()
ppc_files = ppc_directory["sessFile"].unique().tolist()
include_files = list(set(rsc_files) | set(ppc_files))

session_files = pursuit.file_reader.load_session_files(data_dir2, include_files=include_files)


In [ ]:
#convert extracted pursuit session file data into dataframes
session_dfs = pursuit.df_utils.convert_sessions_to_dfs(session_files)

In [ ]:
#filter for valid cells by region using region directory dfs
rsc_session_dfs = pursuit.df_utils.filter_spkTables(rsc_directory, session_dfs)
ppc_session_dfs = pursuit.df_utils.filter_spkTables(ppc_directory, session_dfs)

In [ ]:
#build region dfs
rsc_df = pursuit.df_utils.build_region_df(rsc_directory, rsc_session_dfs)
ppc_df = pursuit.df_utils.build_region_df(ppc_directory, ppc_session_dfs)

In [ ]:
#verify sessFile, spkTable group counts (neurons) 
pursuit.df_utils.count_neurons(rsc_df)

In [ ]:
#verify sessFile, spkTable group counts (neurons) 
pursuit.df_utils.count_neurons(ppc_df)

In [ ]:
#save files to parquet
pursuit.df_utils.save_to_parquet(rsc_df, filename="RSC_sessions")
pursuit.df_utils.save_to_parquet(ppc_df, filename="PPC_sessions")